# **Road Pothole Detection and Segementation with Road Damage Assesement**

**Connection to the Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls /content/drive/MyDrive/Machine_Vision/


In [ ]:
!ls /content/drive/MyDrive/Machine_Vision/New_Pothole_Dataset/


In [ ]:
# Installing the Ultralytics: Essential for running YOLOv8 segmentation tailored for pothole detection
!pip install ultralytics --quiet

**Importing Necessary Libraries**

In [ ]:
# Importing essential libraries for YOLOv8 pothole segmentation project
# Core Python Libraries
import random
import os     # Handles operating system-related tasks such as file and directory operations.
import shutil # Used for high-level file operations like copying, moving, or deleting files and directories
import warnings     #  Suppress any unnecessary warnings
warnings.filterwarnings('ignore')
import locale # Handles locale settings (such as language, number formatting, and currency). Used here to ensure correct string formatting.
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

os.environ['LC_ALL'] = 'en_US.UTF-8'
os.environ['LANG'] = 'en_US.UTF-8'

# Computer Vision and Image Processing
import cv2  # Used for reading, writing, processing, and manipulating images
import numpy as np
from PIL import Image # A Python Imaging Library alternative for opening, editing, and saving images.

from ultralytics import YOLO  #YOLO framework. Allows to load and train YOLO models
from collections import deque # A double-ended queue data structure from Python’s standard library, useful for efficiently managing a sliding window of detections or predictions

# File and Path Handling
import yaml   # Used for reading and writing configuration files
import pandas as pd
from pathlib import Path  # A way to handle file paths, making it easier to work with directories
import glob   # A library for finding file paths using pattern matching (e.g., finding all .jpg or .png files in a directory)

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Video Processing and Display
from IPython.display import Video   # Enables displaying videos within Jupyter Notebooks

# Customize Seaborn plot vizualization
sns.set(rc={'axes.facecolor': '#e0f7fa'}, style='darkgrid')


**Load Model**

In [ ]:
# Model Loading....
# YOLOv8n-seg model: Pre-trained for precise segmentation
model = YOLO('yolov8n-seg.pt')

**YAML file Info**

In [ ]:
# Define the dataset directory and construct the YAML file path using pathlib
dataset_dir = Path('/content/drive/MyDrive/Machine_Vision/New_Pothole_Dataset')
config_file_initial = dataset_dir / 'data.yaml'

# Load and display the YAML configuration in a human-readable format
with open(config_file_initial, 'r') as stream:
    config_data = yaml.safe_load(stream)
    print(yaml.dump(config_data, default_flow_style=False))


# **Dataset Information, Preprocessing and Visualizations**

In [ ]:
# Define directories for training, validation, and test images from the dataset folder
train_dir = os.path.join(dataset_dir, 'train', 'images')
val_dir = os.path.join(dataset_dir, 'valid', 'images')
test_dir = os.path.join(dataset_dir, 'test', 'images')

# Initialize counters and sets for storing unique image dimensions
train_count, valid_count, test_count = 0, 0, 0
train_dims, valid_dims, test_dims = set(), set(), set()

In [ ]:
# Process training images
for file in os.listdir(train_dir):
    if file.lower().endswith('.jpg'):
        train_count += 1
        with Image.open(os.path.join(train_dir, file)) as img:
            train_dims.add(img.size)

# Process validation images
for file in os.listdir(val_dir):
    if file.lower().endswith('.jpg'):
        valid_count += 1
        with Image.open(os.path.join(val_dir, file)) as img:
            valid_dims.add(img.size)

# Process test images
for file in os.listdir(test_dir):
    if file.lower().endswith('.jpg'):
        test_count += 1
        with Image.open(os.path.join(test_dir, file)) as img:
            test_dims.add(img.size)

# Display the total counts for each set
print("Total training images:", train_count)
print("Total validation images:", valid_count)
print("Total test images:", test_count)


In [ ]:
# Check whether all training images share the same dimensions
if len(train_dims) == 1:
    print("Every training image is of size:", train_dims.pop())
else:
    print("Training images have various sizes:", train_dims)

# Check the validation images for uniformity in dimensions
if len(valid_dims) == 1:
    print("Every validation image is of size:", valid_dims.pop())
else:
    print("Validation images have different sizes:", valid_dims)

# Check the test images for uniformity in dimensions
if len(test_dims) == 1:
    print("Every test image is of size:", test_dims.pop())
else:
    print("Test images have different sizes:", test_dims)

### Deleting "Object" class to have only the "Pothole Class" in the dataset

In [ ]:
# Define dataset splits and corresponding directories
splits = ['train', 'valid', 'test']

# Loop through each split (train, valid, test)
for split in splits:
    labels_dir = os.path.join(dataset_dir, split, 'labels')
    images_dir = os.path.join(dataset_dir, split, 'images')

    # Check if labels directory exists
    if not os.path.exists(labels_dir):
        print(f"Labels directory not found for {split} split: {labels_dir}")
        continue

    # Loop through each label file in the directory
    for txt_file in os.listdir(labels_dir):
        if not txt_file.endswith('.txt'):
            continue

        txt_path = os.path.join(labels_dir, txt_file)
        with open(txt_path, 'r') as f:
            lines = f.readlines()

        # Flag to determine if "object" class (class index 1) is present
        remove_file = False
        for line in lines:
            tokens = line.strip().split()
            if tokens and tokens[0] == "1":  # "1" corresponds to the "object" class
                remove_file = True
                break

        if remove_file:
            # Delete the label file
            print(f"Deleting label file: {txt_path}")
            os.remove(txt_path)

            # Assume the corresponding image has the same base name and a .jpg extension
            base_name = os.path.splitext(txt_file)[0]
            image_path = os.path.join(images_dir, base_name + '.jpg')
            if os.path.exists(image_path):
                print(f"Deleting image file: {image_path}")
                os.remove(image_path)
            else:
                print(f"Image file not found for {txt_file} in {images_dir}")


In [ ]:
def count_images_in_dir(directory, extension='.jpg'):
    count = 0
    for file in os.listdir(directory):
        if file.lower().endswith(extension):
            count += 1
    return count

# Define directories for images in each split
train_dir = os.path.join(dataset_dir, 'train', 'images')
val_dir = os.path.join(dataset_dir, 'valid', 'images')
test_dir = os.path.join(dataset_dir, 'test', 'images')

# Re-count the images after deletion
train_count_after = count_images_in_dir(train_dir)
val_count_after = count_images_in_dir(val_dir)
test_count_after = count_images_in_dir(test_dir)

print("After deletion:")
print("Total training images:", train_count_after)
print("Total validation images:", val_count_after)
print("Total test images:", test_count_after)


### Removing images and annotation files that have mismatches

In [ ]:
# Define the labels directory (adjust this path as needed)
labels_dir = os.path.join(dataset_dir, 'train', 'labels')

# List to keep track of files with annotation issues
mismatch_files = []

# Iterate over each label file in the directory
for filename in os.listdir(labels_dir):
    if filename.endswith('.txt'):
        filepath = os.path.join(labels_dir, filename)
        with open(filepath, 'r') as f:
            lines = f.readlines()
            for idx, line in enumerate(lines):
                parts = line.strip().split()
                # Check that there's at least one class ID and 6 coordinates (minimum for a triangle polygon)
                if len(parts) < 7:
                    print(f"Annotation mismatch in file '{filename}' on line {idx+1}: Not enough elements for a valid segmentation polygon.")
                    mismatch_files.append(filename)
                    break  # Stop checking further lines in this file
                # Check that the number of coordinates (excluding the class ID) is even
                if (len(parts) - 1) % 2 != 0:
                    print(f"Annotation mismatch in file '{filename}' on line {idx+1}: The number of coordinates is not even.")
                    mismatch_files.append(filename)
                    break

if not mismatch_files:
    print("No annotation mismatches found.")
else:
    print("Files with annotation mismatches:", set(mismatch_files))


In [ ]:
# Define the labels directory (adjust this path as needed)
labels_dir = os.path.join(dataset_dir, 'test', 'labels')

# List to keep track of files with annotation issues
mismatch_files = []

# Iterate over each label file in the directory
for filename in os.listdir(labels_dir):
    if filename.endswith('.txt'):
        filepath = os.path.join(labels_dir, filename)
        with open(filepath, 'r') as f:
            lines = f.readlines()
            for idx, line in enumerate(lines):
                parts = line.strip().split()
                # Check that there's at least one class ID and 6 coordinates (minimum for a triangle polygon)
                if len(parts) < 7:
                    print(f"Annotation mismatch in file '{filename}' on line {idx+1}: Not enough elements for a valid segmentation polygon.")
                    mismatch_files.append(filename)
                    break  # Stop checking further lines in this file
                # Check that the number of coordinates (excluding the class ID) is even
                if (len(parts) - 1) % 2 != 0:
                    print(f"Annotation mismatch in file '{filename}' on line {idx+1}: The number of coordinates is not even.")
                    mismatch_files.append(filename)
                    break

if not mismatch_files:
    print("No annotation mismatches found.")
else:
    print("Files with annotation mismatches:", set(mismatch_files))


In [ ]:
# Define the labels directory (adjust this path as needed)
labels_dir = os.path.join(dataset_dir, 'valid', 'labels')

# List to keep track of files with annotation issues
mismatch_files = []

# Iterate over each label file in the directory
for filename in os.listdir(labels_dir):
    if filename.endswith('.txt'):
        filepath = os.path.join(labels_dir, filename)
        with open(filepath, 'r') as f:
            lines = f.readlines()
            for idx, line in enumerate(lines):
                parts = line.strip().split()
                # Check that there's at least one class ID and 6 coordinates (minimum for a triangle polygon)
                if len(parts) < 7:
                    print(f"Annotation mismatch in file '{filename}' on line {idx+1}: Not enough elements for a valid segmentation polygon.")
                    mismatch_files.append(filename)
                    break  # Stop checking further lines in this file
                # Check that the number of coordinates (excluding the class ID) is even
                if (len(parts) - 1) % 2 != 0:
                    print(f"Annotation mismatch in file '{filename}' on line {idx+1}: The number of coordinates is not even.")
                    mismatch_files.append(filename)
                    break

if not mismatch_files:
    print("No annotation mismatches found.")
else:
    print("Files with annotation mismatches:", set(mismatch_files))


In [ ]:
# Define your dataset directories
dataset_dir = '/content/drive/MyDrive/Machine_Vision/New_Pothole_Dataset'
labels_dir = os.path.join(dataset_dir, 'train', 'labels')
images_dir = os.path.join(dataset_dir, 'train', 'images')

# Set of label filenames (mismatched annotations) to remove
mismatched_files = {
    'images84_jpg.rf.8ff54df740279ec03e068c581535e3da.txt', 'images84_jpg.rf.f55527784995caff51379f9308787797.txt'
}

for txt_filename in mismatched_files:
    # Construct full path for the label file
    label_file_path = os.path.join(labels_dir, txt_filename)

    # Remove the label file if it exists
    if os.path.exists(label_file_path):
        os.remove(label_file_path)
        print(f"Deleted label file: {label_file_path}")
    else:
        print(f"Label file not found: {label_file_path}")

    # Construct corresponding image filename by replacing .txt with .jpg
    image_filename = txt_filename.replace('.txt', '.jpg')
    image_file_path = os.path.join(images_dir, image_filename)

    # Remove the corresponding image file if it exists
    if os.path.exists(image_file_path):
        os.remove(image_file_path)
        print(f"Deleted image file: {image_file_path}")
    else:
        print(f"Image file not found: {image_file_path}")


In [ ]:
# Define your dataset directories
dataset_dir = '/content/drive/MyDrive/Machine_Vision/New_Pothole_Dataset'
labels_dir = os.path.join(dataset_dir, 'test', 'labels')
images_dir = os.path.join(dataset_dir, 'test', 'images')

# Set of label filenames (mismatched annotations) to remove
mismatched_files = {
    'images84_jpg.rf.a5560d7daa1adda2732b77141d683a8d.txt'
}



for txt_filename in mismatched_files:
    # Construct full path for the label file
    label_file_path = os.path.join(labels_dir, txt_filename)

    # Remove the label file if it exists
    if os.path.exists(label_file_path):
        os.remove(label_file_path)
        print(f"Deleted label file: {label_file_path}")
    else:
        print(f"Label file not found: {label_file_path}")

    # Construct corresponding image filename by replacing .txt with .jpg
    image_filename = txt_filename.replace('.txt', '.jpg')
    image_file_path = os.path.join(images_dir, image_filename)

    # Remove the corresponding image file if it exists
    if os.path.exists(image_file_path):
        os.remove(image_file_path)
        print(f"Deleted image file: {image_file_path}")
    else:
        print(f"Image file not found: {image_file_path}")


### Final Dataset Image counts

In [ ]:
# Define directories for images in each split
train_dir = os.path.join(dataset_dir, 'train', 'images')
val_dir = os.path.join(dataset_dir, 'valid', 'images')
test_dir = os.path.join(dataset_dir, 'test', 'images')

# Re-count the images after deletion
train_count_after = count_images_in_dir(train_dir)
val_count_after = count_images_in_dir(val_dir)
test_count_after = count_images_in_dir(test_dir)

print("After deletion:")
print("Total training images:", train_count_after)
print("Total validation images:", val_count_after)
print("Total test images:", test_count_after)

### Updated YAML File for the Dataset

In [ ]:
# Path to your existing YAML file
yaml_file_path = os.path.join(dataset_dir, 'data.yaml')

# Load the current YAML file
with open(yaml_file_path, 'r') as f:
    data = yaml.safe_load(f)

# Modify the YAML: remove the "object" class and update number of classes to 1
data['names'] = ['Pothole']
data['nc'] = 1

# Save the updated configuration to a new file called new_data.yaml
new_yaml_file_path = os.path.join(dataset_dir, 'new_data.yaml')
with open(new_yaml_file_path, 'w') as f:
    yaml.dump(data, f)

print(f"Updated YAML saved as {new_yaml_file_path}")


In [ ]:
# Define the dataset directory and construct the YAML file path using pathlib
dataset_dir = Path('/content/drive/MyDrive/Machine_Vision/New_Pothole_Dataset')
config_file = dataset_dir / 'new_data.yaml'

# Load and display the YAML configuration in a human-readable format
with open(config_file, 'r') as stream:
    config_data = yaml.safe_load(stream)
    print(yaml.dump(config_data, default_flow_style=False))

In [ ]:
# Randomly select and display 12 training images in a 3x4 grid layout
# Create a list of JPEG files from the training directory
train_files = [f for f in os.listdir(train_dir) if f.lower().endswith('.jpg')]
random.seed(0)
sample_files = random.sample(train_files, 12)

plt.figure(figsize=(20, 18))
for idx, file in enumerate(sample_files):
    img = Image.open(os.path.join(train_dir, file))
    plt.subplot(3, 4, idx + 1)
    plt.imshow(img)
    plt.axis('off')

plt.suptitle('Randomly Selected Training Images', fontsize=20)
plt.tight_layout()
plt.show()
del train_files

In [ ]:
# Define the path to the training labels directory
labels_path = os.path.join(dataset_dir, 'train', 'labels')
pothole_counts = []
total_potholes = 0  # Initialize a counter for total potholes

# Iterate through label files and count pothole instances per image
for label_file in os.listdir(labels_path):
    if label_file.endswith('.txt'):
        with open(os.path.join(labels_path, label_file), 'r') as f:
            lines = f.readlines()
            pothole_counts.append(len(lines))
            total_potholes += len(lines)  # Accumulate the count

# Print the total number of potholes in the dataset
print(f"Total number of potholes in the training dataset: {total_potholes}")

# Plot a histogram showing the distribution of pothole counts per image
plt.figure(figsize=(8, 5))
plt.hist(pothole_counts, bins=range(0, max(pothole_counts) + 2), align='left', color='skyblue', edgecolor='black')
plt.title('Distribution of Pothole Instances per Image')
plt.xlabel('Number of Potholes')
plt.ylabel('Frequency')
plt.show()



In [ ]:
# Example image and label file paths
image_path = os.path.join(dataset_dir, 'train', 'images', 'pic-40-_jpg.rf.5dad084d8bfbd13fae7fe0c6b5083048.jpg')
label_path = os.path.join(dataset_dir, 'train', 'labels', 'pic-40-_jpg.rf.5dad084d8bfbd13fae7fe0c6b5083048.txt')

# Read the image and convert from BGR to RGB for displaying purposes
image = cv2.imread(image_path)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
height, width, _ = image.shape

# Open and parse the label file to extract the polygon coordinates for one pothole
with open(label_path, 'r') as file:
    lines = file.readlines()

# Process the first annotation (one pothole) for demonstration
line = lines[0].strip()
parts = line.split()
# The first element is the class id (0 for potholes), the rest are normalized polygon coordinates
polygon_coords = list(map(float, parts[1:]))

# Convert normalized coordinates to actual pixel positions
points = []
for i in range(0, len(polygon_coords), 2):
    x_norm, y_norm = polygon_coords[i], polygon_coords[i + 1]
    x = int(x_norm * width)
    y = int(y_norm * height)
    points.append([x, y])
points = np.array(points, dtype=np.int32)

# Create a mask for the pothole region using the polygon points
mask = np.zeros((height, width), dtype=np.uint8)
cv2.fillPoly(mask, [points], 255)

# Extract the pothole region from the image using the mask
pothole_region = cv2.bitwise_and(image, image, mask=mask)

# Convert the pothole region to grayscale and apply Canny edge detection
pothole_gray = cv2.cvtColor(pothole_region, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(pothole_gray, threshold1=50, threshold2=150)

# Plot the original image, the pothole region, and the detected edges
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(image_rgb)
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 3, 2)
# Convert BGR to RGB for displaying the pothole region
plt.imshow(cv2.cvtColor(pothole_region, cv2.COLOR_BGR2RGB))
plt.title('Pothole Region')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(edges, cmap='gray')
plt.title('Edge Detection (Canny)')
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Print a summary of the YOLOv8 model architecture
model.info()  # Displays model info and architecture details

In [ ]:
print(model.model)

# **Model Traning**

In [ ]:
# Training the YOLOv8n model using selected Hyperparameters
model_results = model.train(
    data=config_file,     # Path to the dataset configuration
    epochs=150,             # Number of training epochs
    imgsz=640,            # Input image size (square)
    patience=15,          # Early stopping patience based on validation loss
    batch=16,             # Number of images per batch
    optimizer='AdamW',    # Selected Optimizer
    lr0=0.0001,            # Selected initial learning rate
    lrf=0.01,             # Final learning rate multiplier (lr0 * lrf)
    momentum=0.9,         # Chosen momentum value
    dropout=0.25,          # Dropout rate for regularization
    device=0,             # Device to run the training on (e.g., GPU index)
    seed=42               # Seed for reproducibility
)

In [ ]:
# Directories to check
dirs = ["runs/segment/train"]

for directory in dirs:
    print(f"\nContents of {directory}:\n")
    if os.path.exists(directory):
        files = os.listdir(directory)
        for file in files:
            print(file)
    else:
        print("Directory not found.")

In [ ]:
# Define source and destination paths
source_dirs = ["runs/segment/train"]
destination = "/content/drive/MyDrive/Machine_Vision/New_Pothole_Dataset/results"

# Create the destination directory if it doesn't exist
os.makedirs(destination, exist_ok=True)

# Move the directories to Google Drive
for src in source_dirs:
    if os.path.exists(src):
        dst = os.path.join(destination, os.path.basename(src))
        shutil.move(src, dst)
        print(f"Moved {src} to {dst}")
    else:
        print(f"Source directory {src} not found.")


# **Model Performance Evaluation - Training & Validation**

**Post-training result images**

In [ ]:
# Specify the directory containing post-training result images
results_dir = '/content/drive/MyDrive/Machine_Vision/New_Pothole_Dataset/results/train'

# Construct the complete path for the results image file
results_img_path = os.path.join(results_dir, 'results.png')

# Function to display the training/validation results image
def show_results(image_path):
    # Load the image in BGR format and convert to RGB for accurate color rendering
    image_bgr = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    # Create a figure with a custom background color and size
    fig, ax = plt.subplots(figsize=(20, 8), facecolor='#f9f9f9')
    ax.imshow(image_rgb)

    ax.set_title('Loss Trends: Training and Validation', fontsize=20, color='navy')
    ax.axis('off')
    plt.tight_layout()
    plt.show()

show_results(results_img_path)


In [ ]:
# Construct path and load CSV data
results_csv_path = os.path.join(results_dir, 'results.csv')
df = pd.read_csv(results_csv_path)
df.columns = df.columns.str.strip()

# Plot training and validation bounding box loss
plt.figure(figsize=(8, 2))
plt.plot(df['epoch'], df['train/box_loss'], label='Train Box Loss', color='teal', linewidth=2)
plt.plot(df['epoch'], df['val/box_loss'], label='Val Box Loss', color='coral', linestyle='--', linewidth=2)
plt.title('Bounding Box Loss Learning Curve', fontsize=12)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim(0, 2)
plt.legend()
plt.show()


In [ ]:
# Plot training and validation classification loss
plt.figure(figsize=(8, 2))
plt.plot(df['epoch'], df['train/cls_loss'], label='Train Cls Loss', color='purple', linewidth=2)
plt.plot(df['epoch'], df['val/cls_loss'], label='Val Cls Loss', color='orange', linestyle='--', linewidth=2)
plt.title('Classification Loss Learning Curve', fontsize=12)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim(0, 2)
plt.legend()
plt.show()


In [ ]:
# Plot training and validation DFL loss
plt.figure(figsize=(8, 2))
plt.plot(df['epoch'], df['train/dfl_loss'], label='Train DFL Loss', color='darkgreen', linewidth=2)
plt.plot(df['epoch'], df['val/dfl_loss'], label='Val DFL Loss', color='red', linestyle='--', linewidth=2)
plt.title('Distribution Focal Loss Learning Curve', fontsize=12)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim(0, 2)
plt.legend()
plt.show()


In [ ]:
# Plot training and validation segmentation loss
plt.figure(figsize=(8, 2))
plt.plot(df['epoch'], df['train/seg_loss'], label='Train Seg Loss', color='navy', linewidth=2)
plt.plot(df['epoch'], df['val/seg_loss'], label='Val Seg Loss', color='magenta', linestyle='--', linewidth=2)
plt.title('Segmentation Loss Learning Curve', fontsize=12)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim(0, 5)
plt.legend()
plt.show()


In [ ]:
# Define a function to load an image and convert its color from BGR to RGB
def load_rgb_image(path):
    img = cv2.imread(path)
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Create lists of tuples for 'Box' and 'Mask' metrics with filenames and titles
box_plots = [
    ('BoxP_curve.png', 'Bounding Box Precision-Confidence Curve'),
    ('BoxR_curve.png', 'Bounding Box Recall-Confidence Curve'),
    ('BoxF1_curve.png', 'Bounding Box F1-Confidence Curve')
]

mask_plots = [
    ('MaskP_curve.png', 'Mask Precision-Confidence Curve'),
    ('MaskR_curve.png', 'Mask Recall-Confidence Curve'),
    ('MaskF1_curve.png', 'Mask F1-Confidence Curve')
]

# Set up a new figure with a specified size
plt.figure(figsize=(10, 10))

# There are 6 images in total; arrange them in a 3x2 grid using plt.subplot
for idx, (filename, title) in enumerate(box_plots):
    img_path = os.path.join(results_dir, filename)
    img = load_rgb_image(img_path)
    plt.subplot(3, 2, 2 * idx + 1)  # Place in the first column of each row
    plt.imshow(img)
    plt.title(title, fontsize=12,color='darkgreen')
    plt.axis('off')

for idx, (filename, title) in enumerate(mask_plots):
    img_path = os.path.join(results_dir, filename)
    img = load_rgb_image(img_path)
    plt.subplot(3, 2, 2 * idx + 2)  # Place in the second column of each row
    plt.imshow(img)
    plt.title(title, fontsize=12, color='darkblue')
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
def display_metric_curves(metric_dict, folder, grid_dims=(1, 2)):
    rows, cols = grid_dims
    fig = plt.figure(figsize=(10, 5))

    # Iterate over each metric image
    for idx, (img_file, caption) in enumerate(metric_dict.items()):
        # Build the full image path
        path = os.path.join(folder, img_file)

        # Load the image using cv2 and convert its color space from BGR to RGB
        loaded_img = cv2.imread(path)
        converted_img = cv2.cvtColor(loaded_img, cv2.COLOR_BGR2RGB)

        # Add a subplot for the current image
        ax = fig.add_subplot(rows, cols, idx + 1)
        ax.imshow(converted_img)
        ax.set_title(caption, fontsize=12, color='darkred')
        ax.axis('off')

    # Adjust the layout and display the figure
    plt.tight_layout()
    plt.show()

# Example usage with 'Box' and 'Mask' Precision-Recall curves:
pr_metrics = {
    'BoxPR_curve.png': 'Bounding Box Precision-Recall Curve',
    'MaskPR_curve.png': 'Mask Precision-Recall Curve'
}

display_metric_curves(pr_metrics, results_dir, grid_dims=(1, 2))


In [ ]:
def show_confusion_images(raw_img_path, norm_img_path, size=(18, 9)):
    # Load the raw image and convert its color from BGR to RGB
    raw_image = cv2.imread(raw_img_path)
    raw_rgb = cv2.cvtColor(raw_image, cv2.COLOR_BGR2RGB)

    # Load the normalized image and convert its color from BGR to RGB
    norm_image = cv2.imread(norm_img_path)
    norm_rgb = cv2.cvtColor(norm_image, cv2.COLOR_BGR2RGB)

    # Create a figure and add two subplots manually
    fig = plt.figure(figsize=size)

    ax_raw = fig.add_subplot(1, 2, 1)
    ax_raw.imshow(raw_rgb)
    ax_raw.set_title('Confusion Matrix Overview', fontsize=12, color='darkred')
    ax_raw.axis('off')

    ax_norm = fig.add_subplot(1, 2, 2)
    ax_norm.imshow(norm_rgb)
    ax_norm.set_title('Normalized Confusion Matrix Overview', fontsize=12, color='darkblue')
    ax_norm.axis('off')

    plt.tight_layout()
    plt.show()

# Define file paths for the confusion matrix images
raw_confusion_path = os.path.join(results_dir, 'confusion_matrix.png')
normalized_confusion_path = os.path.join(results_dir, 'confusion_matrix_normalized.png')

# Call the function to display the images
show_confusion_images(raw_confusion_path, normalized_confusion_path)


# **Model Evalution**

In [ ]:
def load_and_evaluate_model(results_folder, weight_file, dataset='test'):
    # Construct the full path to the best model weights
    weight_path = os.path.join(results_folder, 'weights', weight_file)

    # Load the model using the best weights
    model_instance = YOLO(weight_path)

    # Evaluate the model on the given dataset split
    evaluation_results = model_instance.val(split=dataset)
    return evaluation_results

# Define the directory containing your results and the best weights filename
results_directory = results_dir
best_weights = 'best.pt'

# Load and evaluate the model; store evaluation metrics
eval_metrics = load_and_evaluate_model(results_directory, best_weights, dataset='test')

In [ ]:
# Transform the evaluation metrics dictionary into a DataFrame
# Using pd.Series to convert the dictionary, then converting it into a DataFrame

filtered_results = {k: v for k, v in eval_metrics.results_dict.items() if k.startswith('metrics/')}

# Convert the results into a DataFrame
metrics_series = pd.Series(filtered_results)
metrics_df = metrics_series.to_frame(name='Metric Value')

# Round the metric values for a cleaner display and print the table
print(metrics_df.round(3))

In [ ]:
# Directories to check
dirs_2 = ["runs/segment/val"]

for directory in dirs_2:
    print(f"\nContents of {directory}:\n")
    if os.path.exists(directory):
        files = os.listdir(directory)
        for file in files:
            print(file)
    else:
        print("Directory not found.")


In [ ]:
# Define source and destination paths
source_dirs_2 = ["runs/segment/val"]
destination_2 = "/content/drive/MyDrive/Machine_Vision/New_Pothole_Dataset/results"

# Create the destination directory if it doesn't exist
os.makedirs(destination_2, exist_ok=True)

# Move the directories to Google Drive
for src in source_dirs_2:
    if os.path.exists(src):
        dst = os.path.join(destination_2, os.path.basename(src))
        shutil.move(src, dst)
        print(f"Moved {src} to {dst}")
    else:
        print(f"Source directory {src} not found.")


**Confusion Matrix**

In [ ]:
results_dir2 = '/content/drive/MyDrive/Machine_Vision/New_Pothole_Dataset/results/val'

# Define file paths for the confusion matrix images
raw_confusion_path_test = os.path.join(results_dir2, 'confusion_matrix.png')
normalized_confusion_path_test = os.path.join(results_dir2, 'confusion_matrix_normalized.png')

# Call the function to display the images
show_confusion_images(raw_confusion_path_test, normalized_confusion_path_test)


In [ ]:
# Create lists of tuples for 'Box' and 'Mask' metrics with filenames and titles
box_plots = [
    ('BoxP_curve.png', 'Bounding Box Precision-Confidence Curve'),
    ('BoxR_curve.png', 'Bounding Box Recall-Confidence Curve'),
    ('BoxF1_curve.png', 'Bounding Box F1-Confidence Curve')
]

mask_plots = [
    ('MaskP_curve.png', 'Mask Precision-Confidence Curve'),
    ('MaskR_curve.png', 'Mask Recall-Confidence Curve'),
    ('MaskF1_curve.png', 'Mask F1-Confidence Curve')
]

# Set up a new figure with a specified size
plt.figure(figsize=(10, 10))

# There are 6 images in total; arrange them in a 3x2 grid using plt.subplot
for idx, (filename, title) in enumerate(box_plots):
    img_path = os.path.join(results_dir2, filename)
    img = load_rgb_image(img_path)
    plt.subplot(3, 2, 2 * idx + 1)  # Place in the first column of each row
    plt.imshow(img)
    plt.title(title, fontsize=12,color='darkgreen')
    plt.axis('off')

for idx, (filename, title) in enumerate(mask_plots):
    img_path = os.path.join(results_dir, filename)
    img = load_rgb_image(img_path)
    plt.subplot(3, 2, 2 * idx + 2)  # Place in the second column of each row
    plt.imshow(img)
    plt.title(title, fontsize=12, color='darkblue')
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Example usage with 'Box' and 'Mask' Precision-Recall curves:
pr_metrics = {
    'BoxPR_curve.png': 'Bounding Box Precision-Recall Curve',
    'MaskPR_curve.png': 'Mask Precision-Recall Curve'
}

display_metric_curves(pr_metrics, results_dir2, grid_dims=(1, 2))


**Model Inferencing : Test Data**

In [ ]:
# Define the full path to the best model weights file
best_model_path = os.path.join(results_directory, 'weights', best_weights)

# Create a YOLO model instance using the best weights
best_model = YOLO(best_model_path)

def display_test_inferences(test_dir, model_instance, num_samples=9, img_size=640):
    # Retrieve a sorted list of JPEG images in the test directory
    all_images = sorted([f for f in os.listdir(test_dir) if f.lower().endswith('.jpg')])
    total_imgs = len(all_images)

    # Evenly select image indices (if total_imgs < num_samples, use all images)
    if total_imgs >= num_samples:
        indices = np.linspace(0, total_imgs - 1, num=num_samples, dtype=int)
    else:
        indices = np.arange(total_imgs)

    selected_imgs = [all_images[i] for i in indices]

    # Create a figure with a 3x3 grid for displaying results
    fig = plt.figure(figsize=(8, 9))
    fig.suptitle('Testing Inference Results', fontsize=24, color='darkgreen')

    # Loop through each selected image, run inference, and plot the annotated output
    for idx, img_name in enumerate(selected_imgs):
        img_full_path = os.path.join(test_dir, img_name)
        inference = model_instance.predict(source=img_full_path, imgsz=img_size)
        # Get the annotated image (assumed to be in BGR) and convert to RGB for display
        annotated_bgr = inference[0].plot()
        annotated_rgb = cv2.cvtColor(annotated_bgr, cv2.COLOR_BGR2RGB)

        ax = fig.add_subplot(3, 3, idx + 1)
        ax.imshow(annotated_rgb)
        ax.axis('off')

    plt.tight_layout()
    plt.show()

# Define the test images directory path
testing_dir = os.path.join(dataset_dir, 'test', 'images')

# Call the function to display 9 evenly spaced test inferences using the best_model
display_test_inferences(testing_dir, best_model, num_samples=9, img_size=640)

**Additional Evaluation Metrics : Average IoU and Dice Score**

In [ ]:
# Load Ground Truth Mask

def load_ground_truth_mask(img_path, labels_folder):
    # Derive label filename from the image filename
    base_name = os.path.splitext(os.path.basename(img_path))[0]
    label_path = os.path.join(labels_folder, base_name + '.txt')

    # Read the image to get its dimensions
    image = cv2.imread(img_path)
    if image is None:
        print(f"Warning: Could not read image at {img_path}")
        return None
    height, width, _ = image.shape

    # Initialize a blank mask
    mask = np.zeros((height, width), dtype=np.uint8)

    # If the label file doesn't exist, return an all-zero mask (no pothole)
    if not os.path.exists(label_path):
        return mask

    # Parse each line (each polygon) and draw it on the mask
    with open(label_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            # The first part is the class ID, the rest are polygon coords
            polygon_coords = list(map(float, parts[1:]))
            points = []
            for i in range(0, len(polygon_coords), 2):
                x_norm, y_norm = polygon_coords[i], polygon_coords[i + 1]
                x = int(x_norm * width)
                y = int(y_norm * height)
                points.append([x, y])
            points = np.array(points, dtype=np.int32)
            cv2.fillPoly(mask, [points], 255)

    return mask

# Load Ground Truth Bounding Boxes

def load_ground_truth_boxes(img_path, labels_folder):
    base_name = os.path.splitext(os.path.basename(img_path))[0]
    label_path = os.path.join(labels_folder, base_name + '.txt')
    image = cv2.imread(img_path)
    if image is None:
        print(f"Warning: Could not read image at {img_path}")
        return []
    height, width, _ = image.shape
    gt_boxes = []
    if not os.path.exists(label_path):
        return gt_boxes

    with open(label_path, 'r') as f:
        lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            # Skip class id (parts[0]) and get polygon coordinates
            polygon_coords = list(map(float, parts[1:]))
            points = []
            for i in range(0, len(polygon_coords), 2):
                x_norm = polygon_coords[i]
                y_norm = polygon_coords[i+1]
                x = int(x_norm * width)
                y = int(y_norm * height)
                points.append([x, y])
            points = np.array(points, dtype=np.int32)
            # Compute bounding rectangle from the polygon
            x, y, w, h = cv2.boundingRect(points)
            gt_boxes.append([x, y, x+w, y+h])
    return gt_boxes


# Compute IoU and Dice/F1 for a Single Pair of Masks

def compute_iou_dice(pred_mask, gt_mask):
    # Convert masks to binary
    pred_bin = (pred_mask > 0).astype(np.uint8)
    gt_bin   = (gt_mask   > 0).astype(np.uint8)

    intersection = np.sum(pred_bin * gt_bin)
    union = np.sum((pred_bin + gt_bin) > 0)

    iou = intersection / union if union != 0 else 0.0

    # Dice coefficient (equivalent to F1 in segmentation)
    denom = np.sum(pred_bin) + np.sum(gt_bin)
    dice = (2.0 * intersection) / denom if denom != 0 else 0.0

    return iou, dice


# Compute IoU for Two Boxes

def compute_box_iou(boxA, boxB):
    # Boxes are [x1, y1, x2, y2]
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    union = boxAArea + boxBArea - interArea
    iou = interArea / union if union != 0 else 0.0
    return iou

# Evaluate Metrics for Boxes and Masks Across the Test Set

def evaluate_metrics(model, test_images_dir, test_labels_dir, conf=0.5, imgsz=640):
    image_files = glob.glob(os.path.join(test_images_dir, '*.jpg'))

    box_iou_list = []
    box_f1_list = []
    mask_iou_list = []
    mask_f1_list = []

    for img_path in image_files:
        # Load ground truth mask and boxes
        gt_mask = load_ground_truth_mask(img_path, test_labels_dir)
        if gt_mask is None:
            continue
        gt_boxes = load_ground_truth_boxes(img_path, test_labels_dir)

        # Run model inference to get predicted masks and boxes
        results = model.predict(source=img_path, imgsz=imgsz, conf=conf)
        result = results[0]

        # Evaluate Masks

        pred_mask = np.zeros_like(gt_mask)
        if result.masks is not None:
            mask_data = result.masks.data.cpu().numpy()  # shape: [N, H, W]
            for m in mask_data:
                pred_mask[m > 0] = 255
        m_iou, m_dice = compute_iou_dice(pred_mask, gt_mask)
        mask_iou_list.append(m_iou)
        mask_f1_list.append(m_dice)  # Dice is equivalent to F1 score for segmentation

        # Evaluate Boxes

        pred_boxes = []
        if hasattr(result, 'boxes') and result.boxes is not None:
            # Extract predicted boxes in [x1, y1, x2, y2] format
            pred_boxes_array = result.boxes.xyxy.cpu().numpy()  # shape: [N, 4]
            for box in pred_boxes_array:
                pred_boxes.append(box)

        # Greedy matching between predicted boxes and ground truth boxes
        matches = []   # store IoU for each matched pair
        used_preds = set()
        for gt_box in gt_boxes:
            best_iou = 0
            best_idx = -1
            for i, pred_box in enumerate(pred_boxes):
                if i in used_preds:
                    continue
                iou_val = compute_box_iou(gt_box, pred_box)
                if iou_val > best_iou:
                    best_iou = iou_val
                    best_idx = i
            if best_iou >= 0.5:  # Consider as a true positive if IoU >= 0.5
                matches.append(best_iou)
                used_preds.add(best_idx)

        # Calculate precision, recall, F1 for boxes
        TP = len(matches)
        FP = len(pred_boxes) - TP
        FN = len(gt_boxes) - TP
        box_f1 = (2 * TP) / (2 * TP + FP + FN) if (2 * TP + FP + FN) > 0 else 0.0
        box_mean_iou = np.mean(matches) if matches else 0.0

        box_iou_list.append(box_mean_iou)
        box_f1_list.append(box_f1)

    # Compute mean values over all images
    mean_box_iou = np.mean(box_iou_list) if box_iou_list else 0.0
    mean_box_f1 = np.mean(box_f1_list) if box_f1_list else 0.0
    mean_mask_iou = np.mean(mask_iou_list) if mask_iou_list else 0.0
    mean_mask_f1 = np.mean(mask_f1_list) if mask_f1_list else 0.0

    return mean_box_iou, mean_box_f1, mean_mask_iou, mean_mask_f1


In [ ]:
test_images_dir = os.path.join(dataset_dir, 'test', 'images')
test_labels_dir = os.path.join(dataset_dir, 'test', 'labels')

mean_box_iou, mean_box_f1, mean_mask_iou, mean_mask_f1 = evaluate_metrics(best_model, test_images_dir, test_labels_dir)
print(f"Box IoU: {mean_box_iou:.3f}, Box F1: {mean_box_f1:.3f}")
print(f"Mask IoU: {mean_mask_iou:.3f}, Mask F1: {mean_mask_f1:.3f}")


**Infecencing : Test Video**

In [ ]:
# Specify the path to sample video
sample_vid = '/content/drive/MyDrive/Machine_Vision/New_Pothole_Dataset/sample_video.mp4'

# Run inference on the sample video using the best model and save the output results
best_model.predict(source=sample_vid, save=True)

In [ ]:
# Directories to check
dirs_3 = ["runs/segment/predict"]

for directory in dirs_3:
    print(f"\nContents of {directory}:\n")
    if os.path.exists(directory):
        files = os.listdir(directory)
        for file in files:
            print(file)
    else:
        print("Directory not found.")


In [ ]:
# Define source and destination paths
source_dirs_3 = ["runs/segment/predict"]
destination_3 = "/content/drive/MyDrive/Machine_Vision/New_Pothole_Dataset/results"

# Create the destination directory if it doesn't exist
os.makedirs(destination_3, exist_ok=True)

# Move the directories to Google Drive
for src in source_dirs_3:
    if os.path.exists(src):
        dst = os.path.join(destination_3, os.path.basename(src))
        shutil.move(src, dst)
        print(f"Moved {src} to {dst}")
    else:
        print(f"Source directory {src} not found.")


In [ ]:
%%bash
export LC_ALL=en_US.UTF-8
export LANG=en_US.UTF-8
ffmpeg -i "/content/drive/MyDrive/Machine_Vision/New_Pothole_Dataset/results/predict/sample_video.avi" \
-c:v libx264 -preset fast -crf 22 -c:a aac -strict experimental \
"/content/drive/MyDrive/Machine_Vision/New_Pothole_Dataset/results/predict/sample_video.mp4"


In [ ]:
# Embed the converted video in the notebook for playback
DisplayVideo = Video("/content/drive/MyDrive/Machine_Vision/New_Pothole_Dataset/results/predict/sample_video.mp4",
                     embed=True, width=600, height=400)
DisplayVideo

In [ ]:
# Specify the directory containing whole dataset
dataset_dir = Path('/content/drive/MyDrive/Machine_Vision/New_Pothole_Dataset')

# Specify the directory containing post-training result images
results_dir = '/content/drive/MyDrive/Machine_Vision/New_Pothole_Dataset/results/train'
best_weights = 'best.pt'
# Define the full path to the best model weights file
best_model_path = os.path.join(results_dir, 'weights', best_weights)

# Create a YOLO model instance using the best weights
best_model = YOLO(best_model_path)


In [ ]:
# Exporting the model
best_model.export(format='onnx')

# **Real-Time Pothole Damage Assement**

In [ ]:
def display_inference_and_masks(testing_dir, sample_idx, yolo_model, img_size=640, conf_thresh=0.5):
    # Get the list of JPEG images in the testing folder
    test_imgs = [fname for fname in os.listdir(testing_dir) if fname.lower().endswith('.jpg')]

    # Select the image at the given index
    chosen_img = test_imgs[sample_idx]
    full_img_path = os.path.join(testing_dir, chosen_img)

    # Run prediction using the YOLO model
    pred_results = yolo_model.predict(source=full_img_path, imgsz=img_size, conf=conf_thresh)

    # Get the annotated image and convert BGR to RGB for display
    annotated = pred_results[0].plot()
    annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)

    # Count the number of segmentation masks (if any)
    mask_plots = len(pred_results[0].masks.data) if pred_results[0].masks is not None else 0
    total_plots = 1 + mask_plots

    # Create a row of subplots for the annotated image and each mask
    fig, axes = plt.subplots(1, total_plots, figsize=(15, 5))
    if total_plots == 1:  # Ensure axes is iterable when there's only one plot
        axes = [axes]

    # Display the main annotated image
    axes[0].imshow(annotated_rgb)
    axes[0].set_title('Annotated Output')
    axes[0].axis('off')

    # Display each segmentation mask if available
    if pred_results[0].masks is not None:
        mask_array = pred_results[0].masks.data.cpu().numpy()
        for idx, mask in enumerate(mask_array):
            binary_mask = (mask > 0).astype(np.uint8) * 255
            axes[idx + 1].imshow(binary_mask, cmap='gray')
            axes[idx + 1].set_title(f'Segmentation Mask {idx + 1}')
            axes[idx + 1].axis('off')

    plt.tight_layout()
    plt.show()

    return pred_results

# Helper function to classify overall road damage severity based on overall damage percentage
def classify_severity(damage_percentage):
    if damage_percentage < 5:
        return "Minor"
    elif damage_percentage < 15:
        return "Moderate"
    else:
        return "Severe"

def analyze_and_display_mask_areas(prediction_result):
    # Ensure that masks exist in the prediction result
    if prediction_result.masks is None:
        print("No segmentation masks available in the prediction result.")
        return

    # Convert the masks tensor to a NumPy array
    mask_data = prediction_result.masks.data.cpu().numpy()

    # Calculate the total number of pixels (assuming all masks are from the same image dimensions)
    img_height, img_width = mask_data.shape[1], mask_data.shape[2]
    total_pixels = img_height * img_width

    total_mask_area = 0  # Accumulator for all mask areas
    pothole_areas = []   # List to store each pothole area

    # Create subplots for each mask for visualization
    num_masks = mask_data.shape[0]
    fig, axes = plt.subplots(1, num_masks, figsize=(12, 8))
    if num_masks == 1:
        axes = [axes]

    # Process each mask to compute area and draw contours
    for idx, mask in enumerate(mask_data):
        # Binarize the mask
        binary = (mask > 0).astype(np.uint8) * 255
        # Convert to BGR for drawing contours
        mask_color = cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)
        # Find contours (if there are multiple contours, sum their areas)
        contours, _ = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        mask_area = 0
        for cnt in contours:
            mask_area += cv2.contourArea(cnt)
        pothole_areas.append(mask_area)
        total_mask_area += mask_area

        # Draw all contours in green on the mask image
        cv2.drawContours(mask_color, contours, -1, (0, 255, 0), 3)
        axes[idx].imshow(mask_color)
        axes[idx].set_title(f'Pothole {idx+1}')
        axes[idx].axis('off')

    plt.tight_layout()
    plt.show()

    # Print individual pothole areas
    for i, area in enumerate(pothole_areas):
        damage_pct = (area / total_pixels) * 100
        print(f"Pothole {i+1} Area: {area:.2f} pixels ({damage_pct:.2f}% of image)")

    # Print overall statistics
    overall_damage_pct = (total_mask_area / total_pixels) * 100
    overall_severity = classify_severity(overall_damage_pct)

    print("-" * 50)
    print(f"Total Pothole Area: {total_mask_area:.2f} pixels")
    print(f"Total Image Area: {total_pixels} pixels")
    print(f"Overall Road Damage: {overall_damage_pct:.2f}%")
    print(f"Overall Damage Severity: {overall_severity}")


In [ ]:
# Define the directory containing validation images
testing_images_directory = os.path.join(dataset_dir, 'test', 'images')

# Run inference and display results for the image at index 25; store the prediction result
predicted_results = display_inference_and_masks(testing_images_directory, sample_idx=15, yolo_model=best_model, img_size=640, conf_thresh=0.5)

# Analyze and display mask areas (with severity classification) using the first prediction result
analyze_and_display_mask_areas(predicted_results[0])


In [ ]:
def process_video_damage(video_input, video_output, yolo_model, frame_resize=640, confidence=0.25, smoothing_window=3):
    # Annotation settings
    text_font = cv2.FONT_HERSHEY_SIMPLEX
    text_scale = 1
    text_origin = (40, 80)
    text_color = (255, 255, 255)   # White text
    text_bg_color = (0, 0, 255)      # Red background for text

    # Initialize a fixed-length deque to store recent damage percentages for smoothing
    damage_history = deque(maxlen=smoothing_window)

    # Open the input video stream
    cap = cv2.VideoCapture(video_input)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Setup video writer with the XVID codec
    codec = cv2.VideoWriter_fourcc(*'XVID')
    writer = cv2.VideoWriter(video_output, codec, 20.0, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Run the YOLO model on the current frame
        inference = yolo_model.predict(source=frame, imgsz=frame_resize, conf=confidence)
        # Obtain the annotated frame
        annotated_frame = inference[0].plot(boxes=True)

        # Initialize damage area for the current frame
        current_damage_area = 0.0
        total_pixels = frame.shape[0] * frame.shape[1]
        mask_details = []  # List to store individual mask details

        # If segmentation masks exist, compute the area for each detected mask
        if inference[0].masks is not None:
            mask_data = inference[0].masks.data.cpu().numpy()
            for i, mask in enumerate(mask_data):
                # Convert mask to a binary image
                binary_img = (mask > 0).astype(np.uint8) * 255
                # Find contours; sum the area of all contours in this mask
                cnts, _ = cv2.findContours(binary_img, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                mask_area = 0.0
                for cnt in cnts:
                    mask_area += cv2.contourArea(cnt)
                current_damage_area += mask_area
                mask_pct = (mask_area / total_pixels) * 100
                mask_details.append((i+1, mask_area, mask_pct))

        # Compute the damage percentage for the current frame
        current_damage = (current_damage_area / total_pixels) * 1000

        # Append current frame's damage percentage to the smoothing window and compute the average
        damage_history.append(current_damage)
        avg_damage = sum(damage_history) / len(damage_history)

        # Determine overall severity based on the averaged damage percentage
        severity_label = classify_severity(avg_damage)

        # Print per-frame detailed mask information
        for idx, area, pct in mask_details:
            print(f"Pothole {idx} Area: {area:.2f} pixels ({pct:.2f}% of image)")
        print("-" * 50)
        print(f"Total Pothole Area: {current_damage_area:.2f} pixels")
        print(f"Total Image Area: {total_pixels} pixels")
        print(f"Overall Road Damage: {current_damage:.2f}%")
        print(f"Overall Damage Severity: {severity_label}")
        print()

        # Draw a background line for the annotation text
        cv2.line(annotated_frame, (text_origin[0], text_origin[1] - 10),
                 (text_origin[0] + 350, text_origin[1] - 10), text_bg_color, 40)
        # Overlay the road damage percentage (using averaged value) on the frame
        cv2.putText(annotated_frame, f'Road Damage: {avg_damage:.2f}%', text_origin,
                    text_font, text_scale, text_color, 2, cv2.LINE_AA)
        # Overlay the overall damage severity on the frame
        severity_origin = (text_origin[0], text_origin[1] + 40)
        cv2.putText(annotated_frame, f'Damage Severity: {severity_label}', severity_origin,
                    text_font, text_scale, text_color, 2, cv2.LINE_AA)

        # Write the annotated frame to the output video file
        writer.write(annotated_frame)

    # Release video capture and writer objects
    cap.release()
    writer.release()

# Example usage:
video_in_path = '/content/drive/MyDrive/Machine_Vision/New_Pothole_Dataset/sample_video.mp4'
video_out_path = '/content/drive/MyDrive/Machine_Vision/New_Pothole_Dataset/road_damage_assessment.avi'
process_video_damage(video_in_path, video_out_path, best_model)


In [ ]:

print(os.path.exists('/content/drive/MyDrive/Machine_Vision/New_Pothole_Dataset/road_damage_assessment.avi'))


In [ ]:
%%bash
export LC_ALL=en_US.UTF-8
export LANG=en_US.UTF-8
ffmpeg -i '/content/drive/MyDrive/Machine_Vision/New_Pothole_Dataset/road_damage_assessment.avi' \
-c:v libx264 -preset fast -crf 22 -c:a aac -strict experimental \
'/content/drive/MyDrive/Machine_Vision/New_Pothole_Dataset/road_damage_assessment_.mp4'

In [ ]:
# Embed and display the video in the notebook

Video_Visual = Video('/content/drive/MyDrive/Machine_Vision/New_Pothole_Dataset/road_damage_assessment_.mp4', embed=True, width=600, height=400)

Video_Visual